In [1]:
    try:
        from pytest import fixture
        from importlib import import_module
        @fixture
        def module(): 
            return import_module(__name__)
    except: ...

In [2]:
    from unittest import TestCase, expectedFailure, skipUnless
    IPY = False
    try:
        from IPython import get_ipython
        from IPython.utils.capture import capture_output
        IPY = get_ipython and get_ipython()
    except: 
        get_ipython = None
    from pathlib import Path
    
    try:
        from importnb import *
        from importnb import load_ipython_extension, unload_ipython_extension
    except:
        # install importnb if it doesn't exist, this makes the tests work on binder.
        __import__('subprocess').check_call("pip install importnb".split())
        from importnb import *

    from functools import partial
    Lazy = partial(Notebook, lazy=True)
    Partial = partial(Notebook, exceptions=BaseException)

    __file__ = globals().get('__file__', 'unittests.ipynb')

In [3]:
    class NullException(BaseException): ...

In [4]:
    class TestExtension(TestCase): 
        def setUp(Test): 
            try:
                from IPython import get_ipython
            except:
                get_ipython = lambda: None
            load_ipython_extension(get_ipython())
                
        def test_import(Test):
            try:
                from . import import_this
            except:
                import import_this

            assert import_this.__file__.endswith('.ipynb')
            assert import_this.test is 42
            assert isinstance(import_this, __import__('types').ModuleType)
            
        @expectedFailure
        def test_failure(Test):
            try:
                from . import failure
            except NullException:
                import failure
            
        def tearDown(Test): unload_ipython_extension()


In [5]:
    class TestContext(TestCase): 
        def setUp(Test): 
            with Notebook(): 
                try:
                    from . import import_this
                except:
                    import import_this
            Test.loader = import_this
                  
        def test_import(Test):
            loader = Test.loader
            assert loader.__file__.endswith('.ipynb')
            assert loader.test is 42
            assert isinstance(loader, __import__('types').ModuleType)
            
            
        def test_reload_with_context(Test):
            with Notebook(): assert reload(Test.loader)

In [6]:
    class TestPartial(TestCase): 
        def setUp(Test): 
            with Partial(stderr=True, stdout=True): 
                try:
                    from . import failure
                except:
                    import failure
            Test.failure = failure
                  
        def test_exception(Test):
            assert Test.failure.a is 42
            assert isinstance(Test.failure._exception, BaseException)
            assert not hasattr(Test.failure, 'b')
            
        def test_traceback(Test):
            from traceback import print_tb
            from io import StringIO
            s = StringIO()
            print_tb(Test.failure._exception.__traceback__, file=s)
            assert """tests/failure.ipynb", line 22, in <module>\n    "    assert False\\n""" in s.getvalue(), """Traceback is not satisfied"""

In [7]:
    class TestRemote(TestCase): 
        @skipUnless(False and IPY
             and (
                Path(__file__).parent / 'PythonDataScienceHandbook').exists(), 
            """requires IP""")
        def test_imports(Test):
            with capture_output(), Notebook():
                try:
                    from .PythonDataScienceHandbook.notebooks import _02_The_Basics_Of_NumPy_Arrays as tutorial
                except:
                    from PythonDataScienceHandbook.notebooks import _02_The_Basics_Of_NumPy_Arrays as tutorial
            assert all(hasattr(tutorial, str) for str in ('grid', 'x', 'x1', 'x2'))

In [8]:
    from pytest import mark

    @mark.skipif(not IPY, reason="""We need ipython""")
    def test_readme():
        get_ipython().system('jupyter nbconvert --to markdown --execute --ExecutePreprocessor.kernel_name=python ../../../readme.ipynb')
        get_ipython().system('cd .. && ipython -m readme')

In [9]:
    if __name__ == '__main__':
        __import__('unittest').main(argv='discover'.split(), exit=False)

unittests.ipynb:5: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
..unittests.ipynb:22: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
xunittests.ipynb:11: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
...s
----------------------------------------------------------------------
Ran 7 tests in 2.087s

OK (skipped=1, expected failures=1)


In [10]:
    if __name__ == '__main__':
        get_ipython().system('python -m pytest ')

============================= test session starts ==============================
platform darwin -- Python 3.5.4, pytest-3.5.1, py-1.5.3, pluggy-0.6.0 -- /Users/tonyfast/anaconda/bin/python
cachedir: ../../../.pytest_cache
benchmark: 3.1.1 (defaults: timer=time.perf_counter disable_gc=False min_rounds=5 min_time=0.000005 max_time=1.0 calibration_precision=10 warmup=False warmup_iterations=100000)
rootdir: /Users/tonyfast/importnb, inifile: tox.ini
plugins: ignore-flaky-0.1.1, forked-0.2, benchmark-3.1.1, importnb-0.3.1
collected 26 items                                                             

test_importnb.ipynb::test_single_file_with_context PASSED                [  3%]
test_importnb.ipynb::test_from_filename PASSED                           [  7%]
test_importnb.ipynb::test_from_execute PASSED                            [ 11%]
test_importnb.ipynb::test_with_doctest PASSED                            [ 15%]
test_importnb.ipynb::test_from_filename_main PASSED                      [